# **Subtask 2: Baseline Model Design & Experimental Framework**
## Jinay Vora - 202201473
## Kunj Kapadiya - 202103018

In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\jinay\OneDrive\Documents\VSCodes\Deep Neural NLP\Project\train .csv')
df.head()

,ID,Tweet,Label
0,3037,ussase jayada dukhad congress ka nichi koti ki...,NO
1,7640,takeout burrito shielded from cold as though i...,YES
2,11848,sight of coworkers' stupid fucking faces endur...,YES
3,4812,TIL that @RayuduAmbati is ek number ka chootiy...,NO
4,13098,porch ceded to bats,YES


In [3]:
df_edited = df.drop('ID', axis=1)
df_edited.head()

,Tweet,Label
0,ussase jayada dukhad congress ka nichi koti ki...,NO
1,takeout burrito shielded from cold as though i...,YES
2,sight of coworkers' stupid fucking faces endur...,YES
3,TIL that @RayuduAmbati is ek number ka chootiy...,NO
4,porch ceded to bats,YES


In [6]:
def encoder(x):
    if x=='NO':
        return 0
    else:
        return 1
        
df_encoded = df_edited.copy()
df_encoded['Label'] = df_encoded['Label'].apply(encoder)
df_encoded

,Tweet,Label
0,ussase jayada dukhad congress ka nichi koti ki...,0
1,takeout burrito shielded from cold as though i...,1
2,sight of coworkers' stupid fucking faces endur...,1
3,TIL that @RayuduAmbati is ek number ka chootiy...,0
4,porch ceded to bats,1
...,...,...
9835,fixin's added to food pyramid,1
9836,yosemite closed indefinitely after bear spotte...,1
9837,description of hot-dog ingredients fails to ru...,1
9838,aides trying to talk trump out of sending asso...,1


In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jinay\AppData\Roaming\nltk_data...


True

In [11]:
import re
from nltk.stem import WordNetLemmatizer

def text_preprocessing(text):
    
    text = re.compile(r'[^a-zA-Z\s]').sub('', text)
    
    text = text.lower()
    
    lemma = WordNetLemmatizer()
    text = ' '.join([lemma.lemmatize(word) for word in text.split()])

    return text

df_preprocessed = df_encoded.copy()
df_preprocessed['Tweet'] = df_preprocessed['Tweet'].apply(text_preprocessing)
df_preprocessed.head()

,Tweet,Label
0,ussase jayada dukhad congress ka nichi koti ki...,0
1,takeout burrito shielded from cold a though it...,1
2,sight of coworkers stupid fucking face endured...,1
3,til that rayuduambati is ek number ka chootiya...,0
4,porch ceded to bat,1


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

features = df_preprocessed['Tweet']
labels = df_preprocessed['Label']

tfidf = TfidfVectorizer()
features = tfidf.fit_transform(features).toarray()

features.shape

(9840, 23445)

In [14]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)